In [1]:
import numpy as np
import pandas as pd

In [2]:
SAMPLE_SHP_IN_DIR = 'data/Shapefile/GDFigure2/SampleArea.shp'
CENSUS_IN_DIR = 'data/GiveDirectly/Survey/GE_HH_Census_2017-07-17_cleanGPS.csv'

In [3]:
df = pd.read_csv(
    CENSUS_IN_DIR,
    usecols=['sublocation_code', 'village_code', 'ge', 'hi_sat', 'treat',
             'latitude', 'longitude', 'elevation', 'accuracy', 'eligible',
             'GPS_imputed', 'dist_from_vill'],
    dtype={
        'sublocation_code': 'Int64',
        'village_code': 'Int64',
        'ge': 'Int32',
        'hi_sat': 'Int32',
        'treat': 'Int32',
        'eligible': 'Int32',
        'GPS_imputed': 'Int32'})

In [4]:
# drop non GE households
df = df.loc[df['ge'] == 1, :]

In [5]:
df

,sublocation_code,village_code,ge,hi_sat,treat,latitude,longitude,elevation,accuracy,eligible,GPS_imputed,dist_from_vill
0,601010101,601010101002,1,0,0,0.134217,34.24749,1154.3391,3.276598,0,0,0.482870
1,601010101,601010101002,1,0,0,0.134323,34.24983,1237.1324,2.936678,0,0,0.558774
2,601010101,601010101002,1,0,0,0.134456,34.24513,1239.7261,3.419073,1,0,0.504906
3,601010101,601010101002,1,0,0,0.135151,34.25025,1193.7246,4.746650,0,0,0.515025
4,601010101,601010101002,1,0,0,0.135537,34.24654,1227.2599,4.197362,1,0,0.340859
...,...,...,...,...,...,...,...,...,...,...,...,...
65380,601050504,601050504009,1,0,0,0.171708,34.31936,1245.6063,4.000000,0,0,0.424235
65381,601050504,601050504009,1,0,0,0.184053,34.32773,1269.8408,4.003346,1,0,1.303420
65382,601050504,601050504009,1,0,0,0.172899,34.31741,1220.9720,6.000000,0,0,0.382637
65383,601050504,601050504009,1,0,0,0.181795,34.31849,1241.3723,2.826617,0,0,0.703957
